In [1]:
import os
import concurrent.futures
import multiprocess as mp
import random


from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
import lightkurve as lk
from astroquery.simbad import Simbad
import pandas as pd 
from numpy import genfromtxt


#tf
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [8]:
d = pd.read_csv('/Users/michaelsong/Documents/GitHub/starzam/Labels/unlabeled_stars.csv',delimiter=',').to_numpy()
print(d)
print(d.shape)

[[1027707 '19253846+3646103' nan ... nan 6 0]
 [1027900 '19254767+3644073' nan ... nan 8 0]
 [1160655 '19232193+3650379' nan ... nan 6 0]
 ...
 [12934848 '19201615+5221593' nan ... nan 7 0]
 [12935035 '19204068+5218589' nan ... nan 8 0]
 [12984227 '19201916+5227134' nan ... nan 6 0]]
(14693, 10)


In [3]:
test_lk = lk.search_lightcurve('KIC '+str(757137), author='Kepler',exptime=1800)[2].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
print(len(test_lk.time.value))
l = random.randrange(250, len(test_lk.time.value))
s = random.randrange(0,len(test_lk.time.value)-l)
print(s,l)
test_lk2 = test_lk.truncate(after=test_lk.time.value[s+l],before=test_lk.time.value[s])
test_periodogram = test_lk2.to_periodogram(frequency = [x for x in np.arange(1,24.5,0.1)])
freq = np.asarray([test_periodogram.frequency,test_periodogram.power*100000])
print(freq[1])

4134
4 4072
[1.79519377 3.161746   1.9126815  1.08549662 2.41231448 3.89998306
 4.89104838 3.11777276 0.77655755 3.80622088 1.16259991 3.49564029
 5.54895021 5.33652764 1.19704182 3.37681438 3.58728056 2.0599733
 0.28076188 2.73299563 1.92169384 2.40103628 1.4361267  1.41031097
 1.16279265 3.12414839 0.46615549 0.88132375 2.16862005 2.29273816
 0.99601452 0.65008921 1.45423652 0.80855058 0.98977803 1.34360667
 1.75448505 0.84434017 0.40883283 0.92919147 1.35235901 0.55030839
 1.18800907 0.76686796 0.16105251 0.55014494 1.21139433 0.55655295
 0.2846481  0.76811317 0.69591169 0.6261121  0.86487284 0.4756488
 0.34212856 0.22099518 0.04710461 0.4151719  0.36631289 0.65788991
 0.32802547 0.30515064 0.82100609 0.27579841 0.47111617 0.23327916
 0.30193204 0.61062661 0.25828905 0.74625896 0.43269537 0.30914628
 0.19224324 0.24481938 0.44861878 0.35021909 0.14322109 0.6183515
 0.15090736 0.36472443 0.39049593 0.09069467 0.12564678 0.48136292
 0.35905368 0.22663526 0.23310776 0.18927219 0.246102

In [5]:
def prepsr(i,sr,j):
    r = False
    if r:
        test_lk = sr[j].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
        l = random.randrange(240, len(test_lk.time.value))
        s = random.randrange(0,len(test_lk.time.value)-l)
        lightcurve = test_lk.truncate(after=test_lk.time.value[s+l],before=test_lk.time.value[s])
    else:
        lightcurve = sr[j].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
    
    periodogram = lightcurve.to_periodogram(frequency = [x for x in np.arange(1,24.5,0.1)])
    # periodogram = periodogram.bin(binsize=10, method='mean')
    freq = np.asarray([periodogram.frequency,periodogram.power*100000])
    # pd.DataFrame(freq[1]).to_csv("/content/drive/MyDrive/Research - Yihong Song and Rohit Prasanna/Yihong/Data/"+str(labeled_data["Kepler_ID"][i])+"_"+str(j)+".csv")
    return np.append(np.append(d[i][0:1],[j]),freq[1])

def getlc(i):
    lc_data = []
    sr = lk.search_lightcurve('KIC '+str(d[i][0]), author='Kepler',exptime=1800)
    # return prepsr(i,sr,1)
    for j in range(len(sr)):
        # x = wrap(prepsr(i,sr,j))
        # if x is not None:
        lc_data.append(prepsr(i,sr,j))
        # y = wrap(external_function_raises)
        # if y is not None:
        #     lc_data.append(y)
    return lc_data

def getdata(start):
    # for k in range(start,int(191449/100)+1,1):
    for k in range(start,start+1,1):
        lc_data = []
        for i in range(k*10,(k+1)*10,1):
            try:
                lc_data = lc_data+getlc(i)
            except:
                continue
            print("\r"+str(i),end="")
        # print(lc_data)
        pd.DataFrame(lc_data).to_csv("/Users/michaelsong/Documents/GitHub/starzam/Data_Unk/"+str(k)+".csv")

In [7]:
getdata(0)

9

In [21]:
# with concurrent.futures.ProcessPoolExecutor() as executor:
#     print(executor.map(getdata, [1,2,3]))

with mp.Pool(8) as pool:
    print(pool.map(getdata, [x for x in range(1447,1449)]))

14479[None, None]
